# Import Library

In [1]:
import os
import fnmatch as fn
import numpy as np
import pandas as pd
import dbfread

import pandas_profiling as pp
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
from matplotlib import font_manager, rc
%matplotlib inline

import descartes
import geopandas as gpd
from shapely.geometry import Point, Polygon
from math import sin,cos,sqrt,atan2,radians 
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.gofplots import qqplot
from scipy.stats import shapiro
from scipy.stats import skew
import tbats

import requests
import logging
import time
import googlemaps
import reverse_geocoder
import pprint
import folium
import zipfile
import json
#%matplotlib.inline
import itertools

from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
import lightgbm as lgb

from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense

matplotlib.rcParams['axes.unicode_minus'] = False # 마이너스 기호도 표시

# 한글 깨짐 방지 목적 #
font_name = font_manager.FontProperties(fname = 'c:/Windows/Fonts/malgun.ttf').get_name()
rc('font',family = font_name)

## 맥
# rc('font', family='/Library/Fonts/AppleGothic.ttf')


import plotly
#!pip install cufflinks
#import plotly.express as px
# Cufflinks wrapper on plotly
import cufflinks as cf
import plotly.graph_objs as go
import plotly.plotly as py
import plotly.figure_factory as ff
from plotly.offline import iplot
from plotly.offline import init_notebook_mode, plot,iplot

init_notebook_mode(connected = True)
cf.go_offline()

import warnings
warnings.filterwarnings(action='ignore') # 경고 메시지를 숨길 때

Using TensorFlow backend.
C:\Users\lions\Anaconda3\lib\site-packages\dask\config.py:168: YAMLLoadWarning:

calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.



# Import Data

In [42]:
df = pd.read_csv('가공데이터/merge_카드_유동_환경_edit1.csv')
df = df1.iloc[:,1:]
df['date'] = pd.to_datetime(df1['date'],format = '%Y-%m-%d')

df_revised = pd.read_csv('가공데이터/merge_카드_유동_환경_edit2.csv')
df_revised = df_revised.iloc[:,1:]
df_revised['date'] = pd.to_datetime(df_revised['date'],format = '%Y-%m-%d')

df_revised2 = pd.read_csv('가공데이터/merge_카드_유동_환경_edit3.csv')
df_revised2 = df_revised2.iloc[:,1:]
df_revised2['date'] = pd.to_datetime(df_revised2['date'],format = '%Y-%m-%d')

# Definition of Problem

- 환경 데이터, 유동 데이터, 그리고 T시점 이전의 카드데이터를 사용하여 동,성별,연령, 카테고리별 카드 소비건수와 카드소비금액를 예측한다.
- 이 경우, 해당 문제는 Sequence Prediction에 해당된다.

## sequence Prediction Classification

- Sequence Prediction은 크게 'One-to-One Model', 'One-to-Many Model', 'Many-to-One model', 'Many-to-Many model'이 있다.
- 이 네 가지 유형들 중에서 현재의 문제는 Multiple Input values를 모델이 받아서 Single Output을 도출하는 구조이므로 'Many-To-one'model에 속한다.
- 우리가 해결할 문제는 Multivariate & Multi-Step Time Series Forecasting Problem임이 자명하다.

In [4]:
df_revised2.head()

,date,year,quarter,month,weekday,day,district,p10_most_freq_day,p10_mean_day,p10_25per,...,81M_55A_M_USE_CNT,40M_60A_M_USE_CNT,70M_60A_M_USE_CNT,71M_60A_M_USE_CNT,80M_60A_M_USE_CNT,81M_60A_M_USE_CNT,40M_65A_M_USE_CNT,70M_65A_M_USE_CNT,80M_65A_M_USE_CNT,81M_65A_M_USE_CNT
0,2018-04-01,2018,2,4,Sunday,1,가회동,나쁨,66.773843,57.656944,...,13.0,61.0,0.0,0.0,66.0,4.0,48.0,0.0,26.0,0.0
1,2018-04-02,2018,2,4,Monday,2,가회동,나쁨,64.771759,56.862500,...,13.0,53.0,9.0,4.0,88.0,0.0,26.0,13.0,44.0,22.0
2,2018-04-03,2018,2,4,Tuesday,3,가회동,나쁨,63.759838,48.220833,...,13.0,48.0,0.0,0.0,123.0,4.0,48.0,0.0,74.0,0.0
3,2018-04-04,2018,2,4,Wednesday,4,가회동,좋음,18.720500,10.300589,...,4.0,57.0,9.0,0.0,57.0,9.0,61.0,4.0,92.0,4.0
4,2018-04-05,2018,2,4,Thursday,5,가회동,좋음,12.597801,10.083333,...,13.0,44.0,4.0,0.0,74.0,0.0,39.0,4.0,83.0,13.0


# Data Preprocessing

## 구체적인 전처리

### 먼저 전처리의 지역 대상은 동으로 한정한다. 왜냐하면 지수 및 모델의 효과가 동을 기준으로 존재하기 때문이다. 

### 다음에 불필요한 변수들을 제거한다. 

### 만약 변수가 정수 또는 부동소수라면 Normalization method 또는 Standardization method를 거친다.

#### Normalization은 범위를 0과 1 사이에 모든 값들을 Rescaling하는 작업이다. Normalization은 관측치의 최대값과 최솟값을 정확하게 추정할 수 있어야 한다.

#### Standardization은 관측치의 평균을 0으로 그리고 분산을 1로 만드는 Rescaling method이다. Standardization은 관측치가 가우시안 분포를 따른다고 전제한다. 다시 말해서, 평균과 표준편차를 정확하게 추정하고 있어야함을 의미한다.

### 범주형 변수 Handling

#### 범주형 변수의 범주는 정수형이 아니다. 가능한 범주가 K개가 있다면 1~k까지의 정수를 부여하는 것이 간단학 쉬어 보인다. 하지만 그 결과 값은 서로에 대해 순서를 매길 수 있기 때문에 범주로서 허용되지 않는다. 몇 가지 대안이 있다.

##### One-hot encoding
    - 모든 비트의 합은 1 --> 선형 종속성 --> 선형 모델의 고유성 조건 깨짐
    
##### Dummy Coding
    - 표현식에서 k-1개만을 사용해서 자유도를 k에서 k-1로 줄인다.

In [45]:
#df3['LENGTH']
df_revised2.columns[:59]

Index(['date', 'year', 'quarter', 'month', 'weekday', 'day', 'district',
       'p10_most_freq_day', 'p10_mean_day', 'p10_25per', 'p10_median_day',
       'p10_75per', 'p25_most_freq_day', 'p25_mean_day', 'p25_25per',
       'p25_median_day', 'p25_75per', 'humi_mean_day', 'humi_median_day',
       'humi_25per', 'humi_75per', 'temp_mean_day', 'temp_median_day',
       'temp_25per', 'temp_75per', 'noise_mean_day', 'noise_median_day',
       'dis_index_most_freq2', 'MAN_0004', 'MAN_0509', 'MAN_1014', 'MAN_1519',
       'MAN_2024', 'MAN_2529', 'MAN_3034', 'MAN_3539', 'MAN_4044', 'MAN_4549',
       'MAN_5054', 'MAN_5559', 'MAN_6064', 'MAN_6569', 'MAN_70U', 'WMAN_0004',
       'WMAN_0509', 'WMAN_1014', 'WMAN_1519', 'WMAN_2024', 'WMAN_2529',
       'WMAN_3034', 'WMAN_3539', 'WMAN_4044', 'WMAN_4549', 'WMAN_5054',
       'WMAN_5559', 'WMAN_6064', 'WMAN_6569', 'WMAN_70U', 'SGNG_NM'],
      dtype='object')

In [38]:
df_revised2.columns[59:185]

Index(['21M_20A_F_USE_AMT', '40M_20A_F_USE_AMT', '50M_20A_F_USE_AMT',
       '70M_20A_F_USE_AMT', '71M_20A_F_USE_AMT', '80M_20A_F_USE_AMT',
       '81M_20A_F_USE_AMT', '21M_25A_F_USE_AMT', '40M_25A_F_USE_AMT',
       '50M_25A_F_USE_AMT',
       ...
       '81M_55A_M_USE_AMT', '40M_60A_M_USE_AMT', '70M_60A_M_USE_AMT',
       '71M_60A_M_USE_AMT', '80M_60A_M_USE_AMT', '81M_60A_M_USE_AMT',
       '40M_65A_M_USE_AMT', '70M_65A_M_USE_AMT', '80M_65A_M_USE_AMT',
       '81M_65A_M_USE_AMT'],
      dtype='object', length=126)

### 첫번째 전처리 작업을 진행한다.

- 시계열 데이터이므로 CV(Cross Validation)은 적용할 수 없다.

In [48]:
def pre_processing1(data):
    
    df_copy = data.copy()
    #df_copy = df_copy[df_copy.columns[59:185]]
    y = []
    y2 = []
    
    for i in np.unique(df_copy['district']):
        df_o = df_copy[df_copy['district'] == i]
 
        df_o.index = df_o['date']
        df_o.drop(['date','quarter','day','SGNG_NM','district'],
                 axis=1,
                 inplace=True)
        
        target = df_revised2.columns[59:185]
        
        x = df_o.loc[:,~(df_o.columns.isin(target))]
        x_t = df_o.loc[:,(df_o.columns.isin(target))]
        
        df_o['year'] = df_o['year'].astype('str')
        df_o['month'] = df_o['month'].astype('str')

        for i in x.columns:
            if ((x[i].dtypes == 'int')|(x[i].dtypes == 'float')):
                if ((skew(x[i]) > -2) & (skew(x[i]) < 2)):
                    x[i] = x[i]
                else:
                    x[i] = np.log(x[i])
                    if ((skew(x[i]) > -2) & (skew(x[i]) < 2)):
                        x[i] = np.exp(x[i])
                        x[i] = np.sqrt(x[i])
            else:
                continue


        x2 = x.loc[:,(x.dtypes == 'int')|(x.dtypes == 'float')]
        normalized_x2 = pd.DataFrame((x2 - x2.mean())/x2.std())
        #normalized_x2.rename(x2.columns,inplace=True)
        #normalized_x2.index = df2.index

        cate = x.loc[:,x.dtypes == 'object'].columns
        df3 = pd.get_dummies(x.loc[:,cate])
        df2 = pd.concat([normalized_x2,df3],axis=1)
        df2 = df2.loc[:,~(df2.columns.isin(cate))]
        df_final = pd.concat([df2,x_t],axis=1)
        y.append(df_final)
        #y2= pd.concat(y[0:len(y)])
    return y

In [50]:
y = pre_processing1(df_revised2)

In [51]:
print(np.unique(df['district']))
print(len(np.unique(df['district'])))
print(len(y))

['가회동' '공릉1동' '공릉2동' '교남동' '부암동' '사직동' '상계10동' '상계1동' '상계2동' '상계3.4동'
 '상계5동' '상계6.7동' '숭인2동' '월계1동' '월계3동' '이화동' '종로1.2.3.4가동' '종로5.6가동'
 '중계2.3동' '중계본동' '창신1동' '창신3동' '청운효자동' '평창동' '하계1동' '혜화동']
26
26


## Convert Sequence into Supervised Problem format

In [52]:
target = df_revised2.columns[59:185]
x_var = y[0].loc[:,~(y[0].columns.isin(target))].columns

In [53]:
def pre_processing2(data,input_steps,output_steps,var):
    x = []
    y = []
    
    data = data.reset_index()
    data = data.dropna(axis='columns')
       
    for i in range(len(data)):
        end_ix = i + input_steps
        output_ix = end_ix + output_steps -1
        
        if (output_ix > len(data)-6):
            break
            
        x_seq = data.loc[i:end_ix-1,:]
        y_seq = data.loc[end_ix:output_ix,var]
        x_seq = x_seq.iloc[:,2:]
     #   y_seq = y_seq.iloc[:,0:]
        
        x_seq = x_seq.values
        y_seq= y_seq.values
        
        x_seq = x_seq.reshape(x_seq.shape)
#        y_seq = y_seq.values
        y_seq = y_seq.reshape(y_seq.shape)
        x.append(x_seq)
        y.append(y_seq)
    return np.array(x),np.array(y)

In [54]:
input_steps = 7
output_steps = 7
var = target[0]
trainx,trainy = pre_processing2(y[0],input_steps,output_steps,var)
trainy = np.array([ab.reshape(1,7).tolist() for ab in trainy]).reshape((-1,7))
print(trainx.shape)
print(trainy.shape)

(347, 7, 301)
(347, 7)


# Modeling


## LSTM State Management

### The Efficiency of learning : 업데이트 전에 얼마만큼의 샘플들을 학습하여야 하는가?

### LSTM 입력층은 3차원이어야 한다.
- 3차원 구성 : [samples, time steps, features]

### The Speed of Learning : 가중치들은 얼마만큼의 업데이트 주기를 가지는가?

### LSTM 내부 상태가 얼마나 자주 reset되어야 하는가?

## Stacked LSTM

In [56]:
def predict_lstm(var):
    
    plate1,plate2 = [], []
    
    for i in range(len(y)):
        input_steps = 7
        output_steps = 7
        
        trainx,trainy = pre_processing2(y[i],input_steps,output_steps,var)
      #  testx,testy = pre_processing2(y2[i],input_steps,output_steps)

        trainy = np.array([ab.reshape(1,7).tolist() for ab in trainy]).reshape((-1,7))
      #  testy = np.array([ab.reshape(1,7).tolist() for ab in testy]).reshape((-1,7))

        n_features = trainx.shape[2]

        # Define model
        model = Sequential()
        model.add(LSTM(150,
                        activation='relu',
                       return_sequences = True,
                        input_shape=(input_steps,n_features)))
        model.add(LSTM(150,
                        activation='relu'))
        model.add(Dense(output_steps))

        # Compile model
        model.compile(optimizer = 'adam', loss = 'mse',metrics = ['accuracy'])

        # fit model
        fit_lstm = model.fit(trainx,
                             trainy,
                             shuffle = False,
                             epochs=100,
                             verbose=1,
                             validation_split = 0.2)
        
        print(fit_lstm.history['loss'])
        print(fit_lstm.history['acc'])
        print(fit_lstm.history['val_loss'])
        print(fit_lstm.history['val_acc'])

In [57]:
predict_lstm(target[0])

W0908 08:45:10.158863 17452 deprecation_wrapper.py:119] From C:\Users\lions\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0908 08:45:10.243637 17452 deprecation_wrapper.py:119] From C:\Users\lions\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0908 08:45:10.274554 17452 deprecation_wrapper.py:119] From C:\Users\lions\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0908 08:45:10.889909 17452 deprecation_wrapper.py:119] From C:\Users\lions\Anaconda3\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0908 08:45:11.180131 17452 deprecation.py:323] From C:\Users\lions\Anaconda3\li

Train on 277 samples, validate on 70 samples
Epoch 1/100
277/277 [==============================] - 2s 7ms/step - loss: 240400.8918 - acc: 0.1408 - val_loss: 262294.3815 - val_acc: 0.2571
Epoch 2/100
277/277 [==============================] - 0s 911us/step - loss: 120951.1650 - acc: 0.1769 - val_loss: 191088.1660 - val_acc: 0.2286
Epoch 3/100
277/277 [==============================] - 0s 936us/step - loss: 134954.1726 - acc: 0.2166 - val_loss: 168733.2560 - val_acc: 0.0857
Epoch 4/100
277/277 [==============================] - 0s 821us/step - loss: 92421.8069 - acc: 0.1227 - val_loss: 143595.9221 - val_acc: 0.1143
Epoch 5/100
277/277 [==============================] - 0s 882us/step - loss: 57499.0016 - acc: 0.1372 - val_loss: 143235.7053 - val_acc: 0.1571
Epoch 6/100
277/277 [==============================] - 0s 828us/step - loss: 52392.7250 - acc: 0.1805 - val_loss: 141577.8068 - val_acc: 0.1714
Epoch 7/100
277/277 [==============================] - 0s 792us/step - loss: 47865.5561 - 

Train on 277 samples, validate on 70 samples
Epoch 1/100
277/277 [==============================] - 2s 8ms/step - loss: 3306451.7175 - acc: 0.1372 - val_loss: 3941280.1071 - val_acc: 0.1857
Epoch 2/100
277/277 [==============================] - 0s 807us/step - loss: 2052833.0478 - acc: 0.0975 - val_loss: 3283113.9429 - val_acc: 0.1571
Epoch 3/100
277/277 [==============================] - 0s 785us/step - loss: 1539616.3051 - acc: 0.1733 - val_loss: 3045144.8143 - val_acc: 0.1857
Epoch 4/100
277/277 [==============================] - 0s 785us/step - loss: 1307598.8114 - acc: 0.1444 - val_loss: 2723308.7536 - val_acc: 0.1143
Epoch 5/100
277/277 [==============================] - 0s 785us/step - loss: 1396117.1922 - acc: 0.1516 - val_loss: 3027614.8357 - val_acc: 0.1286
Epoch 6/100
277/277 [==============================] - 0s 843us/step - loss: 1047535.5907 - acc: 0.1191 - val_loss: 2700543.7714 - val_acc: 0.1857
Epoch 7/100
277/277 [==============================] - 0s 940us/step - loss

Train on 277 samples, validate on 70 samples
Epoch 1/100
277/277 [==============================] - 2s 9ms/step - loss: 1759559.8227 - acc: 0.1408 - val_loss: 1978574.9286 - val_acc: 0.2000
Epoch 2/100
277/277 [==============================] - 0s 947us/step - loss: 1459689.9657 - acc: 0.1444 - val_loss: 1209928.8839 - val_acc: 0.1000
Epoch 3/100
277/277 [==============================] - 0s 900us/step - loss: 1368521.6733 - acc: 0.1191 - val_loss: 1304023.9429 - val_acc: 0.1143
Epoch 4/100
277/277 [==============================] - 0s 850us/step - loss: 1085896.9418 - acc: 0.1227 - val_loss: 844141.1089 - val_acc: 0.1143
Epoch 5/100
277/277 [==============================] - 0s 835us/step - loss: 981925.9671 - acc: 0.1155 - val_loss: 856792.1750 - val_acc: 0.2000
Epoch 6/100
277/277 [==============================] - 0s 835us/step - loss: 926274.9898 - acc: 0.1336 - val_loss: 969690.2893 - val_acc: 0.1143
Epoch 7/100
277/277 [==============================] - 0s 889us/step - loss: 931

Epoch 57/100
277/277 [==============================] - 0s 832us/step - loss: 102107.3767 - acc: 0.1841 - val_loss: 141725.3920 - val_acc: 0.1857
Epoch 58/100
277/277 [==============================] - 0s 897us/step - loss: 101660.0843 - acc: 0.1697 - val_loss: 144611.6999 - val_acc: 0.1714
Epoch 59/100
277/277 [==============================] - 0s 915us/step - loss: 99592.4670 - acc: 0.1841 - val_loss: 146544.9655 - val_acc: 0.1286
Epoch 60/100
277/277 [==============================] - 0s 961us/step - loss: 101302.5166 - acc: 0.1769 - val_loss: 143452.4829 - val_acc: 0.1714
Epoch 61/100
277/277 [==============================] - 0s 933us/step - loss: 100880.3782 - acc: 0.1733 - val_loss: 144945.8813 - val_acc: 0.1429
Epoch 62/100
277/277 [==============================] - 0s 835us/step - loss: 101389.5218 - acc: 0.1697 - val_loss: 145753.3035 - val_acc: 0.1143
Epoch 63/100
277/277 [==============================] - 0s 839us/step - loss: 100404.8950 - acc: 0.1769 - val_loss: 140752.09

...

Train on 277 samples, validate on 70 samples
Epoch 1/100
277/277 [==============================] - 11s 39ms/step - loss: 3927150.0104 - acc: 0.1588 - val_loss: 389482.2098 - val_acc: 0.1000
Epoch 2/100
277/277 [==============================] - 2s 6ms/step - loss: 1735147.5853 - acc: 0.1552 - val_loss: 452058.6473 - val_acc: 0.1000
Epoch 3/100
277/277 [==============================] - 2s 6ms/step - loss: 2703205.8468 - acc: 0.1336 - val_loss: 510397.2830 - val_acc: 0.2143
Epoch 4/100
277/277 [==============================] - 2s 6ms/step - loss: 3333721.3290 - acc: 0.1661 - val_loss: 555242.4232 - val_acc: 0.0857
Epoch 5/100
277/277 [==============================] - 2s 6ms/step - loss: 1647450.1512 - acc: 0.1516 - val_loss: 402629.0812 - val_acc: 0.2000
Epoch 6/100
277/277 [==============================] - 2s 6ms/step - loss: 1438238.4104 - acc: 0.1372 - val_loss: 200415.3815 - val_acc: 0.2143
Epoch 7/100
277/277 [==============================] - 2s 6ms/step - loss: 1022504.7296 -

Epoch 58/100
277/277 [==============================] - 2s 6ms/step - loss: 115283.5875 - acc: 0.1444 - val_loss: 21152.9622 - val_acc: 0.1000
Epoch 59/100
277/277 [==============================] - 2s 6ms/step - loss: 114793.4089 - acc: 0.1516 - val_loss: 21320.0409 - val_acc: 0.0714
Epoch 60/100
277/277 [==============================] - 2s 6ms/step - loss: 114305.9863 - acc: 0.1444 - val_loss: 21568.5595 - val_acc: 0.0714
Epoch 61/100
277/277 [==============================] - 2s 6ms/step - loss: 114113.6577 - acc: 0.1516 - val_loss: 21985.7780 - val_acc: 0.0714
Epoch 62/100
277/277 [==============================] - 2s 6ms/step - loss: 114186.7346 - acc: 0.1480 - val_loss: 22234.0623 - val_acc: 0.0571
Epoch 63/100
277/277 [==============================] - 2s 6ms/step - loss: 113900.2456 - acc: 0.1300 - val_loss: 21809.7876 - val_acc: 0.0571
Epoch 64/100
277/277 [==============================] - 2s 6ms/step - loss: 113544.3268 - acc: 0.1264 - val_loss: 22454.4640 - val_acc: 0.0857

Train on 277 samples, validate on 70 samples
Epoch 1/100
277/277 [==============================] - 11s 41ms/step - loss: 34657951.4729 - acc: 0.1227 - val_loss: 13761634.6571 - val_acc: 0.1429
Epoch 2/100
277/277 [==============================] - 2s 6ms/step - loss: 11571347.8845 - acc: 0.1372 - val_loss: 8660644.2571 - val_acc: 0.1571
Epoch 3/100
277/277 [==============================] - 2s 6ms/step - loss: 7149766.8736 - acc: 0.1408 - val_loss: 5355912.9000 - val_acc: 0.2000
Epoch 4/100
277/277 [==============================] - 2s 6ms/step - loss: 4272964.2401 - acc: 0.1769 - val_loss: 4528282.0857 - val_acc: 0.2000
Epoch 5/100
277/277 [==============================] - 2s 6ms/step - loss: 4446964.0235 - acc: 0.1697 - val_loss: 3762157.5286 - val_acc: 0.0857
Epoch 6/100
277/277 [==============================] - 2s 6ms/step - loss: 3983211.4639 - acc: 0.1155 - val_loss: 4375765.5857 - val_acc: 0.0857
Epoch 7/100
277/277 [==============================] - 2s 6ms/step - loss: 33935

Train on 277 samples, validate on 70 samples
Epoch 1/100
277/277 [==============================] - 12s 43ms/step - loss: 8015033.7220 - acc: 0.1516 - val_loss: 5442878.5571 - val_acc: 0.1143
Epoch 2/100
277/277 [==============================] - 2s 7ms/step - loss: 4603681.4738 - acc: 0.1227 - val_loss: 3302721.9464 - val_acc: 0.1286
Epoch 3/100
277/277 [==============================] - 2s 6ms/step - loss: 2985744.7216 - acc: 0.1697 - val_loss: 2864212.2107 - val_acc: 0.1286
Epoch 4/100
277/277 [==============================] - 2s 6ms/step - loss: 2450759.6115 - acc: 0.1588 - val_loss: 2207057.5750 - val_acc: 0.1571
Epoch 5/100
277/277 [==============================] - 2s 6ms/step - loss: 2937187.4662 - acc: 0.1155 - val_loss: 2066555.7750 - val_acc: 0.1714
Epoch 6/100
277/277 [==============================] - 2s 6ms/step - loss: 1996922.1864 - acc: 0.1444 - val_loss: 1924462.7795 - val_acc: 0.1429
Epoch 7/100
277/277 [==============================] - 2s 6ms/step - loss: 1836296.

Epoch 57/100
277/277 [==============================] - 2s 6ms/step - loss: 994253.5194 - acc: 0.1552 - val_loss: 1094009.2929 - val_acc: 0.1143
Epoch 58/100
277/277 [==============================] - 2s 6ms/step - loss: 984908.1770 - acc: 0.1661 - val_loss: 1055136.6071 - val_acc: 0.1286
Epoch 59/100
277/277 [==============================] - 2s 6ms/step - loss: 989453.7008 - acc: 0.1805 - val_loss: 1161662.8482 - val_acc: 0.0857
Epoch 60/100
277/277 [==============================] - 2s 6ms/step - loss: 1082360.6123 - acc: 0.1408 - val_loss: 1171826.4754 - val_acc: 0.1714
Epoch 61/100
277/277 [==============================] - 2s 6ms/step - loss: 1009234.0221 - acc: 0.1697 - val_loss: 1080182.8518 - val_acc: 0.1143
Epoch 62/100
277/277 [==============================] - 2s 6ms/step - loss: 1061686.4860 - acc: 0.1877 - val_loss: 1739059.2616 - val_acc: 0.1143
Epoch 63/100
277/277 [==============================] - 2s 6ms/step - loss: 1207467.5539 - acc: 0.1444 - val_loss: 1121316.7902